# AlphaDB Features → ML/VectorBT Example

This notebook shows how to:
1. Export features from TimescaleDB to Parquet
2. Load features for ML training
3. Use features with VectorBT for backtesting
4. Build a simple trading strategy

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import sys

# Add scripts directory to path
sys.path.append('../scripts')
from export_features import FeaturesExporter

# Optional: VectorBT for backtesting
try:
    import vectorbt as vbt
    VBT_AVAILABLE = True
except ImportError:
    print("VectorBT not installed. Backtesting features will be limited.")
    VBT_AVAILABLE = False

# Optional: ML libraries
try:
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import classification_report
    ML_AVAILABLE = True
except ImportError:
    print("Scikit-learn not installed. ML examples will be limited.")
    ML_AVAILABLE = False

plt.style.use('default')
sns.set_palette("husl")

ModuleNotFoundError: No module named 'seaborn'

## 1. Export Features from AlphaDB

In [ ]:
# Initialize the exporter
exporter = FeaturesExporter(output_dir="../data/features")

# Check what data is available
date_range = exporter.get_date_range()
available_symbols = exporter.get_available_symbols()

print(f"Available symbols: {available_symbols}")
print(f"Data range: {date_range['earliest_date']} to {date_range['latest_date']}")
print(f"Total records: {date_range['total_records']:,}")

In [ ]:
# Export ML-ready features (last 30 days)
ml_filepath = exporter.export_ml_ready_features(
    symbols=['BTC/USDT'], 
    days=30,
    target_periods=[1, 5, 15]  # 1min, 5min, 15min targets
)

print(f"ML features exported to: {ml_filepath}")

In [ ]:
# Export VectorBT-ready format
if VBT_AVAILABLE:
    vbt_filepath = exporter.export_vectorbt_ready(
        symbol='BTC/USDT',
        days=90
    )
    print(f"VectorBT data exported to: {vbt_filepath}")

## 2. Load and Explore Features

In [ ]:
# Load ML-ready features
df = pd.read_parquet(ml_filepath)

print(f"Dataset shape: {df.shape}")
print(f"Date range: {df['bucket'].min()} to {df['bucket'].max()}")
print(f"\nColumns: {list(df.columns)}")

In [ ]:
# Feature overview
feature_cols = [col for col in df.columns if not col.startswith('target_') and col not in ['bucket', 'symbol']]
target_cols = [col for col in df.columns if col.startswith('target_')]

print(f"Features ({len(feature_cols)}): {feature_cols[:10]}...")  # Show first 10
print(f"Targets ({len(target_cols)}): {target_cols}")

# Basic statistics
df[feature_cols[:5]].describe()

## 3. Feature Analysis & Visualization

In [ ]:
# Plot key features over time
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('BTC Features Over Time', fontsize=16)

# Price
axes[0,0].plot(pd.to_datetime(df['bucket']), df['close_price'])
axes[0,0].set_title('BTC Price')
axes[0,0].set_ylabel('Price ($)')

# Volatility
axes[0,1].plot(pd.to_datetime(df['bucket']), df['hl_range_pct'] * 100)
axes[0,1].set_title('Intrabar Volatility')
axes[0,1].set_ylabel('HL Range (%)')

# VWAP Gap
axes[1,0].plot(pd.to_datetime(df['bucket']), df['vwap_gap'])
axes[1,0].axhline(y=0, color='red', linestyle='--', alpha=0.5)
axes[1,0].set_title('VWAP Gap')
axes[1,0].set_ylabel('Gap ($)')

# Volume
axes[1,1].plot(pd.to_datetime(df['bucket']), df['volume'])
axes[1,1].set_title('Volume')
axes[1,1].set_ylabel('Volume')

plt.tight_layout()
plt.show()

In [ ]:
# Feature correlation matrix
core_features = ['hl_range_pct', 'vwap_gap', 'parkinson_vol', 'return_5m', 'vol_ma_60', 'close_ma_60']
correlation_matrix = df[core_features].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0)
plt.title('Feature Correlation Matrix')
plt.show()

## 4. Simple ML Model Example

In [ ]:
if ML_AVAILABLE:
    # Prepare data for ML
    feature_cols = [col for col in df.columns 
                   if not col.startswith(('target_', 'bucket', 'symbol')) 
                   and col not in ['vol_regime']  # Skip categorical for now
                   ]
    
    # Use 5-minute return direction as target
    target_col = 'target_5m_sign'
    
    # Remove rows with NaN values
    ml_data = df[feature_cols + [target_col]].dropna()
    
    X = ml_data[feature_cols]
    y = ml_data[target_col]
    
    print(f"ML dataset shape: {X.shape}")
    print(f"Target distribution: {y.value_counts()}")
    
    # Split data (time-aware split)
    split_idx = int(len(X) * 0.8)
    X_train, X_test = X.iloc[:split_idx], X.iloc[split_idx:]
    y_train, y_test = y.iloc[:split_idx], y.iloc[split_idx:]
    
    # Train simple Random Forest
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    
    # Predictions
    y_pred = rf.predict(X_test)
    
    print("\nClassification Report:")
    print(classification_report(y_test, y_pred))
    
    # Feature importance
    feature_importance = pd.DataFrame({
        'feature': feature_cols,
        'importance': rf.feature_importances_
    }).sort_values('importance', ascending=False)
    
    plt.figure(figsize=(12, 6))
    sns.barplot(data=feature_importance.head(15), x='importance', y='feature')
    plt.title('Top 15 Feature Importances')
    plt.show()
    
    print("\nTop 10 Most Important Features:")
    print(feature_importance.head(10))
else:
    print("Scikit-learn not available. Skipping ML example.")

## 5. VectorBT Backtesting Example

In [ ]:
if VBT_AVAILABLE:
    # Load VectorBT-ready data
    vbt_data = pd.read_parquet(vbt_filepath)
    
    print(f"VectorBT data shape: {vbt_data.shape}")
    print(f"Columns: {list(vbt_data.columns)}")
    print(f"Index: {vbt_data.index.name} from {vbt_data.index.min()} to {vbt_data.index.max()}")
    
    # Strategy 1: Volatility Breakout
    # Buy when volatility > 60-period average AND VWAP gap > 0
    vol_threshold = vbt_data['hl_range_pct'].rolling(60).mean()
    vwap_threshold = 0
    
    buy_signals = (
        (vbt_data['hl_range_pct'] > vol_threshold) & 
        (vbt_data['vwap_gap'] > vwap_threshold)
    )
    
    sell_signals = (
        (vbt_data['hl_range_pct'] < vol_threshold) | 
        (vbt_data['vwap_gap'] < vwap_threshold)
    )
    
    # Run backtest
    portfolio = vbt.Portfolio.from_signals(
        vbt_data['close'], 
        buy_signals, 
        sell_signals,
        fees=0.001,  # 0.1% fees
        freq='1T'    # 1-minute frequency
    )
    
    print(f"\n=== Volatility Breakout Strategy Results ===")
    print(f"Total Return: {portfolio.total_return():.2%}")
    print(f"Sharpe Ratio: {portfolio.sharpe_ratio():.2f}")
    print(f"Max Drawdown: {portfolio.max_drawdown():.2%}")
    print(f"Win Rate: {portfolio.trades.win_rate:.2%}")
    print(f"Total Trades: {portfolio.trades.count}")
    
    # Plot results
    fig, axes = plt.subplots(3, 1, figsize=(15, 12))
    
    # Price and signals
    axes[0].plot(vbt_data.index, vbt_data['close'], label='BTC Price', alpha=0.7)
    buy_mask = buy_signals & ~buy_signals.shift(1, fill_value=False)
    sell_mask = sell_signals & ~sell_signals.shift(1, fill_value=False)
    
    axes[0].scatter(vbt_data.index[buy_mask], vbt_data['close'][buy_mask], 
                   color='green', marker='^', s=50, label='Buy')
    axes[0].scatter(vbt_data.index[sell_mask], vbt_data['close'][sell_mask], 
                   color='red', marker='v', s=50, label='Sell')
    axes[0].set_title('BTC Price with Trading Signals')
    axes[0].legend()
    
    # Portfolio value
    axes[1].plot(vbt_data.index, portfolio.value())
    axes[1].set_title('Portfolio Value')
    axes[1].set_ylabel('Portfolio Value ($)')
    
    # Features used in strategy
    axes[2].plot(vbt_data.index, vbt_data['hl_range_pct'] * 100, label='Volatility %', alpha=0.7)
    axes[2].plot(vbt_data.index, vol_threshold * 100, label='Vol Threshold', linestyle='--')
    axes[2].set_title('Volatility Signal')
    axes[2].set_ylabel('Volatility (%)')
    axes[2].legend()
    
    plt.tight_layout()
    plt.show()
    
else:
    print("VectorBT not available. Skipping backtesting example.")

## 6. Export Data for External Use

In [ ]:
# Export different formats for various use cases

# 1. Raw features for custom ML pipelines
raw_filepath = exporter.export_features(
    symbols=['BTC/USDT', 'ETH/USDT'],
    start_date=pd.Timestamp.now() - pd.Timedelta(days=7)  # Last week
)

print(f"Raw features exported: {raw_filepath}")

# 2. Quick CSV export for Excel/R users
csv_path = Path(raw_filepath).with_suffix('.csv')
raw_data = pd.read_parquet(raw_filepath)
raw_data.to_csv(csv_path, index=False)
print(f"CSV version: {csv_path}")

# 3. Summary statistics
summary = raw_data.groupby('symbol').agg({
    'close_price': ['mean', 'std', 'min', 'max'],
    'hl_range_pct': ['mean', 'std', 'max'],
    'volume': ['mean', 'sum'],
    'is_green': 'mean'  # Win rate
})

print("\n=== Weekly Summary Statistics ===")
print(summary)

## 7. Next Steps

This notebook showed you how to:

✅ **Export features** from AlphaDB to Parquet files  
✅ **Load and explore** the feature data  
✅ **Build ML models** using the engineered features  
✅ **Backtest strategies** with VectorBT  
✅ **Export data** in various formats  

### What you can do next:

1. **Scale up ML models**: Try XGBoost, LightGBM, or neural networks
2. **Advanced feature engineering**: Create more sophisticated technical indicators
3. **Multi-symbol strategies**: Build portfolio optimization models
4. **Real-time trading**: Use FastAPI to serve model predictions
5. **Risk management**: Add position sizing and drawdown controls

### Production workflow:
```bash
# Daily feature export
python scripts/export_features.py --incremental --hours 24

# Weekly ML model retraining  
python scripts/export_features.py --ml-ready --days 30

# Monthly full backtest
python scripts/export_features.py --vectorbt --days 90
```

Your AlphaDB feature engineering system is now **production-ready** for quantitative research! 🚀